# Setup Mario

In [2]:
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [9]:
# Setup game
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

In [ ]:
# Create a flag - restart or not
done = True
for step in range(1000000): 
    if done: 
        env.reset()
    # Do random actions
    state, reward, done, info = env.step(env.action_space.sample())
    env.render()
env.close()

# Preprocess Environment

In [ ]:
from gym.wrappers import GrayScaleObservation
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
from matplotlib import pyplot as plt

In [ ]:
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)
env = GrayScaleObservation(env, keep_dim=True)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')

In [ ]:
state = env.reset()
state, reward, done, info = env.step([5])

In [ ]:
plt.figure(figsize=(20,16))
for idx in range(state.shape[3]):
    plt.subplot(1,4,idx+1)
    plt.imshow(state[0][:,:,idx])
plt.show()

# Train the RL Model

In [ ]:
import os 
from stable_baselines3 import PPO # ?Algo
from stable_baselines3.common.callbacks import BaseCallback # *Callback

In [ ]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [ ]:
CHECKPOINT_DIR = './train/' # !Saved model 
LOG_DIR = './logs/' # !Tencerflow log file

In [ ]:
# Setup model saving callback
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [ ]:
# Ai model
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, 
            n_steps=512) 

In [ ]:
model.learn(total_timesteps=1000000, callback=callback) # *Train Ai model

In [ ]:
model.save('thisisatestmodel')

# Test it Out

In [ ]:
# Load model
model = PPO.load('./train/best_model_1000000') # *./train/best_model_5000000

In [ ]:
state = env.reset()


In [ ]:
# Loop through the game
while True: 
    action, _state = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()